In [1]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")
        
from scipy.stats import shapiro
from scipy.stats import probplot
from scipy.stats import ttest_ind, mannwhitneyu
from scipy.stats import chi2_contingency
# from statsmodels.stats.weightstats import zconfint

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

# Trade History Analyser (THA)
Анализатор успешности торговли. На вход принимает историю торгов, скаченную с сайта https://www.EXMO.me в формате .csv

In [2]:
df_first = pd.read_csv('trade-history-2021-11-07.csv')
df_middle = pd.read_csv('trade-history-2022-10-10.csv')
df_last = pd.read_csv('trade-history-2023-07-14.csv')

### ФУНКЦИЯ.  объединяет датасеты, если их несколько

In [3]:
def big_dataset(dataframe_first, dataframe_last):
    ### Concatinations dataframes
    return pd.concat([dataframe_first, dataframe_last], ignore_index=True).drop_duplicates()

In [4]:
big_df_1 = pd.concat([df_middle, df_first], ignore_index=True).drop_duplicates()
print(len(df_first), len(df_middle), len(big_df_1))

big_df = pd.concat([df_last, big_df_1], ignore_index=True).drop_duplicates()
print(len(df_first), len(big_df_1), len(big_df))

9764 9496 17219
9764 17219 21247


In [18]:
big_df = pd.concat([df_last, pd.concat([df_middle, df_first], ignore_index=True).drop_duplicates()], ignore_index=True).drop_duplicates()
len(big_df)

21247

### Посмотрим на имеющиеся у нас даные после объединения датасетов

- **Дата/время** - время когда произошла сделка

- **Trade ID** - идентификационный номер сделки

- **Тип** - тип сделки. всего 2 типа сделок: продажа и покупка

- **Валютная пара** - название валютной пары в которой произошла сделка. На бирже около 130 валютных пар

- **Количество** - количество купленной или проданной валюты

- **Цена** - цена по которой произошла сделка

- **Сумма** - общая сумма сделки

- **Тип комиссии** - 2 типа комиссий: maker, taker

- **Размер комиссии** - общая сумма коммиссии за сделку

- **Комиссия %** - процентная ставка комиссии

In [7]:
big_df.head()

,Дата/время,Trade ID,Тип,Валютная пара,Количество,Цена,Сумма,Тип комиссии,Размер комиссии,Комиссия %
0,01.07.2023 05:17,490916799,sell,EXM_ETH,178.663611,0.000007,0.001190,maker,1.180000e-06,0.1%
1,01.07.2023 01:20,490892980,sell,DOGE_USDT,42.552578,0.070000,2.978680,maker,5.957360e-03,0.2%
2,01.07.2023 01:20,490892977,sell,DOGE_USDT,0.361422,0.070000,0.025300,maker,5.059000e-05,0.2%
3,01.07.2023 01:15,490892774,sell,DOGE_BTC,61.195455,0.000002,0.000141,maker,1.400000e-07,0.1%
4,01.07.2023 01:11,490892444,sell,DOGE_BTC,31.501353,0.000002,0.000070,maker,6.000000e-08,0.1%


In [8]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20929 entries, 0 to 27223
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Дата/время       20929 non-null  object 
 1   Trade ID         20929 non-null  int64  
 2   Тип              20929 non-null  object 
 3   Валютная пара    20929 non-null  object 
 4   Количество       20929 non-null  float64
 5   Цена             20929 non-null  float64
 6   Сумма            20929 non-null  float64
 7   Тип комиссии     20929 non-null  object 
 8   Размер комиссии  20929 non-null  float64
 9   Комиссия %       20929 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 1.8+ MB


# НА ВЫХОДЕ ПОСЛЕ АНАЛИЗАТОРА Я ХОЧУ ПОЛУЧИТЬ:
- Период времени, когда велась торговля
- количество торговых пар в которых совершались сделки
- список торговых пар в которых совершались сделки
- датафрейм по каждой валютной паре
- СВОДНАЯ ТАБЛИЦА. Датафрейм.

# СВОДНАЯ ТАБЛИЦА:
- 1 столбец.  ВАЛЮТНАЯ ПАРА
- 2 столбец.  КОЛИЧЕСТВО СДЕЛОК 
- 3 столбец.  КОЛИЧЕСТВО BUY СДЕЛОК
- 4 столбец.  КОЛИЧЕСТВО SELL СДЕЛОК
- 5 столбец.  КОЛИЧЕСТВО КУПЛЕННОЙ ВАЛЮТЫ
- 6 столбец.  КОЛИЧЕСТВО ПРОДАННОЙ ВАЛЮТЫ
- 7 столбец.  МИНИМАЛЬНАЯ ЦЕНА
- 8 столбец.  МАКСИМАЛЬНАЯ ЦЕНА
- 9 столбец.  КОЛЛИЧЕСТВО СРЕДСТВ НА КОТОРЫЕ ЗАКУПАЛСЯ (Сумма закупки)
- 10 столбец.  КОЛЛИЧЕСТВО СРЕДСТВ КОТОРЫЕ ВЫРУЧИЛ ЗА ПРОДАЖУ (Сумма продажи)
- 11 столбец.  ДАТА ПЕРВОЙ ТОРГОВОЙ ОПЕРАЦИИ
- 12 столбец.  ДАТА ПОСЛЕДНЕЙ ТОРГОВОЙ ОПЕРАЦИИ
- 13 столбец.  ПРОДОЛЖИТЕЛЬНОСТЬ НАБЛЮДЕНИЯ (От первой до последней даты сколько дней, часов. минут?)


### ФУНКЦИЯ. Определение даты самой ранней сделки и самой последней сделки. Подсчёт временного периода торговли

In [19]:
def time_period(dataframe):
    all_dates = dataframe['Дата/время'].tolist()
    start = all_dates[-1]
    end = all_dates[0]
    start_dtm = datetime.datetime.strptime(start, '%d.%m.%Y %H:%M')
    end_dtm = datetime.datetime.strptime(end, '%d.%m.%Y %H:%M')
    print(f'\nПроизведён анализ временного периода торговли: с {start} по {end}'
        

  f'\nчто составляет {end_dtm - start_dtm}')
    

In [38]:
time_period(df_jul_2023)


Произведён анализ временного периода торговли: с 01.07.2023 00:57 по 14.07.2023 17:58
что составляет 13 days, 17:01:00


In [14]:
time_period(df_first)


Произведён анализ временного периода торговли: с 14.04.2021 06:57 по 07.11.2021 07:47
что составляет 207 days, 0:50:00


In [15]:
time_period(df_middle)


Произведён анализ временного периода торговли: с 31.08.2021 17:05 по 10.10.2022 07:08
что составляет 404 days, 14:03:00


In [16]:
time_period(df_last)


Произведён анализ временного периода торговли: с 22.11.2021 21:10 по 02.06.2023 12:11
что составляет 556 days, 15:01:00


### ФУНКЦИЯ. Список всех валютных пар в которых совершались сделки и список валют

In [41]:
def all_pairs_list(dataframe):
    return dataframe['Валютная пара'].unique().tolist()

In [42]:
def currents(pair_list):
    currents_list = []
    for el in pair_list:
        curs = el.split('_')
        for el in curs:
            currents_list.append(el)
    return set(currents_list)

In [43]:
pair_list = all_pairs_list(big_df)

In [44]:
len(currents(pair_list))

16

### количество торговых пар и валют в которых совершались сделки 

In [45]:
len(all_pairs_list(big_df))

19

In [46]:
len(currents(pair_list))

16

### ФУНКЦИЯ. датасет по валютной паре

In [47]:
def df_curr_pair(dataframe, current_pair):
    current = str(current_pair.upper())
    return dataframe.loc[(dataframe['Валютная пара'] == current)]

### Список датасетов по каждой валютной паре

In [51]:
dataframe_pairs_list = []
for el in pair_list:
    dataframe_pairs_list.append(df_curr_pair(big_df, el))

### Cводная таблица 

In [52]:
def time_period_1(dataframe):
    all_dates = dataframe['Дата/время'].tolist()
    start = all_dates[-1]
    end = all_dates[0]
    start_dtm = datetime.datetime.strptime(start, '%d.%m.%Y %H:%M')
    end_dtm = datetime.datetime.strptime(end, '%d.%m.%Y %H:%M')
    return start_dtm, end_dtm
    
    
pair_list = all_pairs_list(big_df)
trade_counts_list = []
buy_count_list = []
sell_count_list = []
all_cur_value = []
buy_cur_value = []
sell_cur_value = []
min_price = []
max_price = []
buy_value = []
sell_value = []
first_date = []
last_date = []
delta_date = []



for el in dataframe_pairs_list:
    trade_counts_list.append(len(el))
    all_cur_value.append(el['Количество'].sum())
    a = el['Тип'].value_counts()
    
    if len(a) == 2:
        buy_count_list.append(a[0])
        sell_count_list.append(a[1])  
        buy_cur_value.append(el[el['Тип'] == 'buy']['Количество'].sum())
        sell_cur_value.append(el[el['Тип'] == 'sell']['Количество'].sum())
        min_price.append(el[el['Тип'] == 'buy']['Цена'].min())
        max_price.append(el[el['Тип'] == 'sell']['Цена'].max())
        buy_value.append(el[el['Тип'] == 'buy']['Сумма'].sum())
        sell_value.append(el[el['Тип'] == 'sell']['Сумма'].sum())
        first_date.append(time_period_1(el)[0])
        last_date.append(time_period_1(el)[1])
        delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])
        
    else:
        if str(el['Тип']) == 'buy':
            buy_count_list.append(len(el))
            sell_count_list.append(0)
            buy_cur_value.append(el['Количество'].sum())
            sell_cur_value.append(0)
            min_price.append(el['Цена'].min())
            max_price.append(el['Цена'].max())
            buy_value.append(el['Сумма'].sum())
            sell_value.append(0)
            first_date.append(time_period_1(el)[0])
            last_date.append(time_period_1(el)[1])
            delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])
        else:
            buy_count_list.append(0)
            sell_count_list.append(len(el))
            buy_cur_value.append(0)
            sell_cur_value.append(el['Количество'].sum())
            min_price.append(el['Цена'].min())
            max_price.append(el['Цена'].max())
            buy_value.append(0)
            sell_value.append(el['Сумма'].sum())
            first_date.append(time_period_1(el)[0])
            last_date.append(time_period_1(el)[1])
            delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])

In [53]:
finish_df = pd.DataFrame({'cur_pair' : pair_list,
                          'trd_count' : trade_counts_list,
                          'sell_count': sell_count_list,
                          'buy_count': buy_count_list,
                          'all_cur_value': all_cur_value,
                          'buy_cur_value': buy_cur_value,
                          'sell_cur_value': sell_cur_value,
                          'min_price': min_price,
                          'max_price': max_price,
                          'buy_value': buy_value,
                          'sell_value': sell_value,
                          'first_date':  first_date, 
                          'last_date':  last_date,
                          'time_period': delta_date
                         },
                           columns=['cur_pair','trd_count', 'sell_count', 'buy_count', 'all_cur_value', 'buy_cur_value', 'sell_cur_value', 'min_price', 'max_price', 'buy_value', 'sell_value', 'first_date', 'last_date', 'time_period'])

In [17]:
finish_df

,cur_pair,trd_count,sell_count,buy_count,all_cur_value,buy_cur_value,sell_cur_value,min_price,max_price,buy_value,sell_value,first_date,last_date,time_period
0,EXM_ETH,634,295,339,135581.184934,73161.089560,62420.095374,6.400000e-06,3.000000e-05,0.583311,0.511313,2021-05-05 22:52:00,2023-07-01 05:17:00,786 days 06:25:00
1,DOGE_USDT,19,8,11,1274.026319,887.232694,386.793625,4.000000e-02,9.000000e-02,65.280000,28.230358,2023-04-07 13:54:00,2023-07-01 01:20:00,84 days 11:26:00
2,DOGE_BTC,280,134,146,6729.375561,3364.167442,3365.208119,2.100000e-06,6.200000e-06,0.011697,0.013775,2022-12-05 09:31:00,2023-07-01 01:15:00,207 days 15:44:00
3,EXM_BTC,1350,644,706,388834.087079,206510.175074,182323.912005,3.900000e-07,2.500000e-06,0.157512,0.143389,2021-04-21 11:51:00,2023-07-01 01:05:00,800 days 13:14:00
4,WXT_BTC,432,190,242,186996.094617,96562.005261,90434.089355,7.000000e-08,5.000000e-07,0.012961,0.014626,2021-07-29 10:10:00,2023-06-30 21:14:00,701 days 11:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,ONT_BTC,1,1,0,23.993149,0.000000,23.993149,2.903000e-05,2.903000e-05,0.000000,0.000697,2021-06-04 10:46:00,2021-06-04 10:46:00,0 days 00:00:00
60,ONG_BTC,13,13,0,222.211205,0.000000,222.211205,2.000000e-05,2.178000e-05,0.000000,0.004740,2021-06-04 10:43:00,2021-06-04 10:44:00,0 days 00:01:00
61,XRP_USDT,2,1,1,17.325071,11.423863,5.901208,7.000000e-01,1.000000e+00,7.996704,5.901208,2021-05-23 12:44:00,2021-05-25 00:02:00,1 days 11:18:00
62,XEM_BTC,3,1,2,37.830499,27.153535,10.676964,4.500000e-06,7.000000e-06,0.000133,0.000075,2021-04-23 02:26:00,2021-05-23 16:45:00,30 days 14:19:00


In [21]:
finish_df[finish_df['buy_cur_value'] <  finish_df['sell_cur_value']]

,cur_pair,trd_count,sell_count,buy_count,all_cur_value,buy_cur_value,sell_cur_value,min_price,max_price,buy_value,sell_value,first_date,last_date,time_period
2,DOGE_BTC,280,134,146,6.729376e+03,3364.167442,3.365208e+03,2.100000e-06,6.200000e-06,0.011697,0.013775,2022-12-05 09:31:00,2023-07-01 01:15:00,207 days 15:44:00
8,XRP_ETH,114,50,64,1.260617e+03,601.229036,6.593884e+02,2.100000e-04,3.200000e-04,0.164236,0.183616,2021-04-18 03:36:00,2023-06-28 19:30:00,801 days 15:54:00
9,DAI_RUB,40,17,23,1.893263e+02,67.230226,1.220961e+02,8.000000e+01,9.500000e+01,5575.388039,10655.878398,2023-04-24 12:41:00,2023-06-28 13:40:00,65 days 00:59:00
22,USDT_RUB,734,342,392,2.317781e+03,1098.211513,1.219570e+03,6.700000e+01,7.900000e+01,80226.958928,90634.205528,2021-04-14 14:24:00,2023-01-22 08:11:00,647 days 17:47:00
23,DOGE_EUR,4,4,0,3.409772e+02,0.000000,3.409772e+02,7.585960e-02,7.728467e-02,0.000000,26.315182,2023-01-19 15:23:00,2023-01-19 15:50:00,0 days 00:27:00
26,ETH_GBP,12,12,0,3.997043e-02,0.000000,3.997043e-02,1.085003e+03,1.293549e+03,0.000000,46.613284,2022-12-05 07:27:00,2022-12-05 07:28:00,0 days 00:01:00
32,BTC_USDX,1,1,0,5.377100e-04,0.000000,5.377100e-04,2.500000e+04,2.500000e+04,0.000000,13.442750,2022-10-14 07:02:00,2022-10-14 07:02:00,0 days 00:00:00
34,ALGO_BTC,1,1,0,1.896508e+02,0.000000,1.896508e+02,1.506000e-05,1.506000e-05,0.000000,0.002856,2022-09-16 10:45:00,2022-09-16 10:45:00,0 days 00:00:00
35,TRX_BTC,2,2,0,1.110000e+02,0.000000,1.110000e+02,3.000000e-06,3.000000e-06,0.000000,0.000333,2022-08-02 08:37:00,2022-08-02 08:37:00,0 days 00:00:00
36,SHIB_UAH,6,6,0,2.288936e+06,0.000000,2.288936e+06,4.566900e-04,4.659000e-04,0.000000,1060.809847,2022-08-02 06:17:00,2022-08-02 06:17:00,0 days 00:00:00


In [35]:
df_all_dates = big_df

# сортировка по годам
df_2021 = df_all_dates[df_all_dates['Дата/время'].str.contains("2021")]
df_2022 = df_all_dates[df_all_dates['Дата/время'].str.contains("2022")]
df_2023 = df_all_dates[df_all_dates['Дата/время'].str.contains("2023")]

# сортировка по месяцам
df_jan_2021 = df_2021[df_2021['Дата/время'].str.contains(".01.2021")]
df_feb_2021 = df_2021[df_2021['Дата/время'].str.contains(".02.2021")]
df_mar_2021 = df_2021[df_2021['Дата/время'].str.contains(".03.2021")]
df_apr_2021 = df_2021[df_2021['Дата/время'].str.contains(".04.2021")]
df_may_2021 = df_2021[df_2021['Дата/время'].str.contains(".05.2021")]
df_jun_2021 = df_2021[df_2021['Дата/время'].str.contains(".06.2021")]
df_jul_2021 = df_2021[df_2021['Дата/время'].str.contains(".07.2021")]
df_avg_2021 = df_2021[df_2021['Дата/время'].str.contains(".08.2021")]
df_sep_2021 = df_2021[df_2021['Дата/время'].str.contains(".09.2021")]
df_oct_2021 = df_2021[df_2021['Дата/время'].str.contains(".10.2021")]
df_nov_2021 = df_2021[df_2021['Дата/время'].str.contains(".11.2021")]
df_dec_2021 = df_2021[df_2021['Дата/время'].str.contains(".12.2021")]

df_jan_2022 = df_2022[df_2022['Дата/время'].str.contains(".01.2022")]
df_feb_2022 = df_2022[df_2022['Дата/время'].str.contains(".02.2022")]
df_mar_2022 = df_2022[df_2022['Дата/время'].str.contains(".03.2022")]
df_apr_2022 = df_2022[df_2022['Дата/время'].str.contains(".04.2022")]
df_may_2022 = df_2022[df_2022['Дата/время'].str.contains(".05.2022")]
df_jun_2022 = df_2022[df_2022['Дата/время'].str.contains(".06.2022")]
df_jul_2022 = df_2022[df_2022['Дата/время'].str.contains(".07.2022")]
df_avg_2022 = df_2022[df_2022['Дата/время'].str.contains(".08.2022")]
df_sep_2022 = df_2022[df_2022['Дата/время'].str.contains(".09.2022")]
df_oct_2022 = df_2022[df_2022['Дата/время'].str.contains(".10.2022")]
df_nov_2022 = df_2022[df_2022['Дата/время'].str.contains(".11.2022")]
df_dec_2022 = df_2022[df_2022['Дата/время'].str.contains(".12.2022")]

df_jan_2023 = df_2023[df_2023['Дата/время'].str.contains(".01.2023")]
df_feb_2023 = df_2023[df_2023['Дата/время'].str.contains(".02.2023")]
df_mar_2023 = df_2023[df_2023['Дата/время'].str.contains(".03.2023")]
df_apr_2023 = df_2023[df_2023['Дата/время'].str.contains(".04.2023")]
df_may_2023 = df_2023[df_2023['Дата/время'].str.contains(".05.2023")]
df_jun_2023 = df_2023[df_2023['Дата/время'].str.contains(".06.2023")]
df_jul_2023 = df_2023[df_2023['Дата/время'].str.contains(".07.2023")]
df_avg_2023 = df_2023[df_2023['Дата/время'].str.contains(".08.2023")]
df_sep_2023 = df_2023[df_2023['Дата/время'].str.contains(".09.2023")]
df_oct_2023 = df_2023[df_2023['Дата/время'].str.contains(".10.2023")]
df_nov_2023 = df_2021[df_2021['Дата/время'].str.contains(".11.2023")]
df_dec_2023 = df_2023[df_2023['Дата/время'].str.contains(".12.2023")]


In [32]:

time_period(df_all_dates)
trades_count(df_all_dates)
enumirated_pairs(df_all_dates)
curr_pair_trade_counts(df_all_dates)


print(f'\nСделки за 2021 год. Общее количество: {len(df_2021)}'
      f'\nЯнварь 2021: {len(df_jan_2021)}'
      f'\nФевраль 2021: {len(df_feb_2021)}'
      f'\nМарт 2021: {len(df_mar_2021)}'
      f'\nАпрель 2021: {len(df_apr_2021)}'
      f'\nМай 2021: {len(df_may_2021)}'
      f'\nИюнь 2021: {len(df_jun_2021)}'
      f'\nИюль 2021: {len(df_jul_2021)}'
      f'\nАвгуст 2021: {len(df_avg_2021)}'
      f'\nСентябрь 2021: {len(df_sep_2021)}'
      f'\nОктябрь 2021: {len(df_oct_2021)}'
      f'\nНоябрь 2021: {len(df_nov_2021)}'
      f'\nДекабрь 2021: {len(df_dec_2021)}')

print(f'\nСделки за 2022 год. Общее количество: {len(df_2022)}'
      f'\nЯнварь 2022: {len(df_jan_2022)}'
      f'\nФевраль 2022: {len(df_feb_2022)}'
      f'\nМарт 2022: {len(df_mar_2022)}'
      f'\nАпрель 2022: {len(df_apr_2022)}'
      f'\nМай 2022: {len(df_may_2022)}'
      f'\nИюнь 2022: {len(df_jun_2022)}'
      f'\nИюль 2022: {len(df_jul_2022)}'
      f'\nАвгуст 2022: {len(df_avg_2022)}'
      f'\nСентябрь 2022: {len(df_sep_2022)}'
      f'\nОктябрь 2022: {len(df_oct_2022)}'
      f'\nНоябрь 2022: {len(df_nov_2022)}'
      f'\nДекабрь 2022: {len(df_dec_2022)}')

print(f'\nСделки за 2023 год. Общее количество: {len(df_2023)}'
      f'\nЯнварь 2023: {len(df_jan_2023)}'
      f'\nФевраль 2023: {len(df_feb_2023)}'
      f'\nМарт 2023: {len(df_mar_2023)}'
      f'\nАпрель 2023: {len(df_apr_2023)}'
      f'\nМай 2023: {len(df_may_2023)}'
      f'\nИюнь 2023: {len(df_jun_2023)}'
      f'\nИюль 2023: {len(df_jul_2023)}'
      f'\nАвгуст 2023: {len(df_avg_2023)}'
      f'\nСентябрь 2023: {len(df_sep_2023)}'
      f'\nОктябрь 2023: {len(df_oct_2023)}'
      f'\nНоябрь 2023: {len(df_nov_2023)}'
      f'\nДекабрь 2023: {len(df_dec_2023)}')

current_pair = input('\nВведите/выбирете название торговой пары для которой будет произведён анализ: ')
print(50 * '_')

# сортировка по имени торговой пары
current_pair_dataset = df_all_dates.loc[(df_all_dates['Валютная пара'] == current_pair)]

time_period(current_pair_dataset)
trades_count_pair(current_pair_dataset)
analitica(current_pair_dataset, current_pair)
moda(current_pair_dataset)
maxmin(current_pair_dataset)
middle(current_pair_dataset)
median(current_pair_dataset)

#####################################
time_period(df_2023)
trades_count(df_2023)
enumirated_pairs(df_2023)
curr_pair_trade_counts(df_2023)
current_pair = input('\nВведите/выбирете название торговой пары для которой будет произведён анализ: ')
print(50 * '_')
current_pair_dataset = df_all_dates.loc[(df_all_dates['Валютная пара'] == current_pair)]
time_period(current_pair_dataset)
trades_count_pair(current_pair_dataset)
analitica(current_pair_dataset, current_pair)
moda(current_pair_dataset)
maxmin(current_pair_dataset)
middle(current_pair_dataset)
median(current_pair_dataset)

########################################


Произведён анализ временного периода торговли: с 14.04.2021 06:57 по 14.07.2023 17:58
что составляет 821 days, 11:01:00


NameError: name 'trades_count' is not defined

In [33]:
import pandas as pd
from scipy import stats
import datetime
import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import warnings
import datetime
import pprint
if not sys.warnoptions:
    warnings.simplefilter("ignore")

# Функция для вывода всех торговых пар, которые нашлись в истории торгов
# пары выводятся пронумированными
def enumirated_pairs(dataframe):
    all_pairs_list = dataframe['Валютная пара'].unique().tolist()
    print(f'\nОбщее число торговых пар в которых совершались сделки: {len(all_pairs_list)}')

# функция выводит сообщение о временном периоде проанализированных данных
# начало торгов - самая ранняя сделка в датафрейме
# конец торгов - самая последняя сделка в датафрейме
def time_period(dataframe):
    all_dates = dataframe['Дата/время'].tolist()
    start = all_dates[-1]
    end = all_dates[0]
    start_dtm = datetime.datetime.strptime(start, '%d.%m.%Y %H:%M')
    end_dtm = datetime.datetime.strptime(end, '%d.%m.%Y %H:%M')
    print(f'\nПроизведён анализ временного периода торговли: с {start} по {end}'
          f'\nчто составляет {end_dtm - start_dtm}')

# Объединение датасетов
def big_dataset(dataframe_first, dataframe_last):
    big_df = pd.concat([dataframe_first, dataframe_last], ignore_index=True)
    big_df_uniq = big_df.drop_duplicates()
    # что то сделать с индексами, т. к после дубликата их значения сбиваются
    return big_df_uniq

# Проверка при объединении датасетов на дубликаты
def duplicates(dataframe_first, dataframe_last):
    big_df = pd.concat([dataframe_first, dataframe_last], ignore_index=True).drop_duplicates()
    if len(big_df) == len(dataframe_first) + len(dataframe_last):
        print('Полное объединение датасетов. Дубликатов нет.\n'
              'Следует проверить последние и первые даты в датасетах\n'
              'возможно есть промежуточный датасет ')
    else:
        print('Объединение датасетов произошло. Число дубликатов:', len(dataframe_first) + len(dataframe_last) - len(big_df))


# Переформатирование типа колонки время в дататайм формат
def to_datetime(dataframe):
    dataframe['Дата/время'] = pd.to_datetime(dataframe['Дата/время'], format='%d.%m.%Y %H:%M')
    return dataframe

# Общее число сделок
# из них сделок BUY и SELL
def trades_count(dataframe):
    print(f'\nОбщее число сделок во всех торговых парах за проанализированный период времени: {len(dataframe)} из них')
    print('BUY сделки:', len(dataframe.loc[(dataframe['Тип'] == 'buy')]))
    print('SELL сделки:', len(dataframe.loc[(dataframe['Тип'] == 'sell')]))

def trades_count_pair(dataframe):
    print(f'\nОбщее число сделок в торговой паре за проанализированный период времени: {len(dataframe)} из них')
    print('BUY сделки:', len(dataframe.loc[(dataframe['Тип'] == 'buy')]))
    print('SELL сделки:', len(dataframe.loc[(dataframe['Тип'] == 'sell')]))

# Визуализация данных по Buy и Sell
def visual_buy_sell(dataframe):
    g = sns.countplot(dataframe['Тип'])
    g.set_xticklabels(['buy','sell'])
    plt.show()

# количество сделок в каждой торговой паре
def curr_pair_trade_counts(dataframe):
    print('Колличество сделок в каждой торговой паре:')
    print(dataframe['Валютная пара'].value_counts())

# Статистика

def moda(dataframe):
    print('\nМОДА')
    print('цена:', stats.mode(dataframe.Цена))  # scipy way
    print('Количество:', stats.mode(dataframe.Количество))  # scipy way
    print('Сумма:', stats.mode(dataframe.Сумма))

def median(dataframe):
    print('\nМЕДИАНА')
    print('цена:', dataframe.Цена.median())  # pandas way
    print('Количество:', dataframe.Количество.median())
    print('Сумма:', dataframe.Сумма.median())

def middle(dataframe):
    print('\nСРЕДНЕЕ')
    print('цена:', dataframe.Цена.mean())  # pandas way
    print('Количество:', dataframe.Количество.mean())
    print('Сумма:', dataframe.Сумма.mean())

def maxmin(dataframe):
    print('\nМАКСИМУМ')
    max = dataframe.Цена.max()
    print('цена:', max)  # pandas way
    print('\nМИНИМУМ')
    print('цена:', dataframe.Цена.min())  # pandas way

# аналитика торговли (применять к одной торговой паре)
def analitica(dataframe, current_pair):
    current_pair_dataset_buy = dataframe.loc[(current_pair_dataset['Тип'] == 'buy')]
    current_pair_dataset_sell = dataframe.loc[(current_pair_dataset['Тип'] == 'sell')]
    currents_list = currents(current_pair)
    print(f'\nминимальная цена по которой покупали: {current_pair_dataset_buy.Цена.min()} '
          f'{currents_list[1]}\n'
          f'максимальная цена по которой продавали: {current_pair_dataset_sell.Цена.max()} '
          f'{currents_list[1]}\n'
          f'oбъём купленной валюты за весь период торгов: {current_pair_dataset_buy.Количество.sum()} '
          f'{currents_list[0]}\n'
          f'oбъём проданной валюты за весь период торгов: {current_pair_dataset_sell.Количество.sum()} '
          f'{currents_list[0]}\n'
          f'потрачено денег за весь период торгов: {current_pair_dataset_buy.Сумма.sum()} '
          f'{currents_list[1]}\n'
          f'заработано денег за весь период торгов: {current_pair_dataset_sell.Сумма.sum()} '
          f'{currents_list[1]}\n\n'
          f'дельта максимальная цена - минимальная: {current_pair_dataset_sell.Цена.max() - current_pair_dataset_buy.Цена.min()} '
          f'{currents_list[1]}\n'
          f'разница объёмов проданный - купленный: {current_pair_dataset_sell.Количество.sum() - current_pair_dataset_buy.Количество.sum()} '
          f'{currents_list[0]}\n'
          f'коэфицент объёмов проданное/купленное: {round(current_pair_dataset_sell.Количество.sum()/current_pair_dataset_buy.Количество.sum(), 2)}\n'
          f'потраченныe деньги минус заработанные: {current_pair_dataset_sell.Сумма.sum() - current_pair_dataset_buy.Сумма.sum()} '
          f'{currents_list[1]}\n'
          f'коэфицент потраченные/заработанные: {round(current_pair_dataset_sell.Сумма.sum()/current_pair_dataset_buy.Сумма.sum(), 2)}\n')

def currents(current_pair):
    pair_list = current_pair.split('_')
    return pair_list

# Проверка на дисбаланс классов
def dissbalance(dataframe):
    # class count
    class_count_0, class_count_1 = dataframe['Тип'].value_counts()

    # Separate class
    class_0 = dataframe[dataframe['Тип'] == 'buy']
    class_1 = dataframe[dataframe['Тип'] == 'sell']# print the shape of the class
    print('buy class 0:', class_0.shape)
    print('sell class 1:', class_1.shape)

# Изменение начального датасета, добавление демипеременных, избавление от типов object
def prepare_data(dataframe):
    prep_data = pd.concat([dataframe, pd.get_dummies(dataframe['Тип'])], axis=1)
    prep_data.drop(columns=['Тип'], inplace=True)
    prep_data = pd.concat([prep_data, pd.get_dummies(dataframe['Тип комиссии'])], axis=1)
    prep_data.drop(columns=['Тип комиссии'], inplace=True)
    prep_data = pd.concat([prep_data, pd.get_dummies(dataframe['Комиссия %'])], axis=1)
    prep_data.drop(columns=['Комиссия %'], inplace=True)
    prep_data.drop(columns=['Trade ID'], inplace=True)
    prepare_data = to_datetime(prep_data)
    return prepare_data
    
# Число сделок в день
def per_day(dataframe):
    new_vals = []
    vals = dataframe['Дата/время'].values
    for el in vals:
        numb = el[0:2]
        new_vals.append(numb)
    dataframe['days'] = new_vals
    dataframe['days'].value_counts()
    return dataframe['days'].value_counts()

#######################################


In [26]:
def time_period_1(dataframe):
    all_dates = dataframe['Дата/время'].tolist()
    start = all_dates[-1]
    end = all_dates[0]
    start_dtm = datetime.datetime.strptime(start, '%d.%m.%Y %H:%M')
    end_dtm = datetime.datetime.strptime(end, '%d.%m.%Y %H:%M')
    return start_dtm, end_dtm
    
    
pair_list = all_pairs_list(df_jun_2023)
trade_counts_list = []
buy_count_list = []
sell_count_list = []
all_cur_value = []
buy_cur_value = []
sell_cur_value = []
min_price = []
max_price = []
buy_value = []
sell_value = []
first_date = []
last_date = []
delta_date = []



for el in dataframe_pairs_list:
    trade_counts_list.append(len(el))
    all_cur_value.append(el['Количество'].sum())
    a = el['Тип'].value_counts()
    
    if len(a) == 2:
        buy_count_list.append(a[0])
        sell_count_list.append(a[1])  
        buy_cur_value.append(el[el['Тип'] == 'buy']['Количество'].sum())
        sell_cur_value.append(el[el['Тип'] == 'sell']['Количество'].sum())
        min_price.append(el[el['Тип'] == 'buy']['Цена'].min())
        max_price.append(el[el['Тип'] == 'sell']['Цена'].max())
        buy_value.append(el[el['Тип'] == 'buy']['Сумма'].sum())
        sell_value.append(el[el['Тип'] == 'sell']['Сумма'].sum())
        first_date.append(time_period_1(el)[0])
        last_date.append(time_period_1(el)[1])
        delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])
        
    else:
        if str(el['Тип']) == 'buy':
            buy_count_list.append(len(el))
            sell_count_list.append(0)
            buy_cur_value.append(el['Количество'].sum())
            sell_cur_value.append(0)
            min_price.append(el['Цена'].min())
            max_price.append(el['Цена'].max())
            buy_value.append(el['Сумма'].sum())
            sell_value.append(0)
            first_date.append(time_period_1(el)[0])
            last_date.append(time_period_1(el)[1])
            delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])
        else:
            buy_count_list.append(0)
            sell_count_list.append(len(el))
            buy_cur_value.append(0)
            sell_cur_value.append(el['Количество'].sum())
            min_price.append(el['Цена'].min())
            max_price.append(el['Цена'].max())
            buy_value.append(0)
            sell_value.append(el['Сумма'].sum())
            first_date.append(time_period_1(el)[0])
            last_date.append(time_period_1(el)[1])
            delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])

,Дата/время,Trade ID,Тип,Валютная пара,Количество,Цена,Сумма,Тип комиссии,Размер комиссии,Комиссия %
9,30.06.2023 21:14,490853754,sell,WXT_BTC,1.375000e+03,9.000000e-08,0.000124,maker,1.200000e-07,0.1%
10,30.06.2023 18:07,490803461,buy,USDT_KZT,1.693411e+00,4.310000e+02,729.859999,maker,5.080230e-03,0.3%
11,30.06.2023 18:07,490803459,buy,USDT_KZT,4.333333e+00,4.320000e+02,1871.999999,maker,1.299999e-02,0.3%
12,30.06.2023 18:07,490803457,buy,USDT_KZT,3.357921e+00,4.330000e+02,1453.979997,maker,1.007376e-02,0.3%
13,30.06.2023 17:41,490795757,buy,EXM_ETH,4.085260e+01,6.600000e-06,0.000270,maker,4.085260e-02,0.1%
...,...,...,...,...,...,...,...,...,...,...
569,01.06.2023 12:17,482572821,sell,EXM_ETH,1.002196e+02,7.200000e-06,0.000722,maker,7.200000e-07,0.1%
570,01.06.2023 11:02,482560741,buy,DOGE_RUB,2.687857e+01,5.600000e+00,150.520000,maker,8.063571e-02,0.3%
571,01.06.2023 07:24,482525644,sell,SHIB_RUB,2.281706e+06,8.000000e-04,1825.364571,maker,5.476094e+00,0.3%
572,01.06.2023 00:21,482457869,buy,EXM_ETH,3.446901e+02,7.100000e-06,0.002447,maker,3.446901e-01,0.1%


In [34]:
def time_period_1(dataframe):
    all_dates = dataframe['Дата/время'].tolist()
    start = all_dates[-1]
    end = all_dates[0]
    start_dtm = datetime.datetime.strptime(start, '%d.%m.%Y %H:%M')
    end_dtm = datetime.datetime.strptime(end, '%d.%m.%Y %H:%M')
    return start_dtm, end_dtm
    
    
pair_list = all_pairs_list(df_jun_2023)
trade_counts_list = []
buy_count_list = []
sell_count_list = []
all_cur_value = []
buy_cur_value = []
sell_cur_value = []
min_price = []
max_price = []
buy_value = []
sell_value = []
first_date = []
last_date = []
delta_date = []



for el in dataframe_pairs_list:
    trade_counts_list.append(len(el))
    all_cur_value.append(el['Количество'].sum())
    a = el['Тип'].value_counts()
    
    if len(a) == 2:
        buy_count_list.append(a[0])
        sell_count_list.append(a[1])  
        buy_cur_value.append(el[el['Тип'] == 'buy']['Количество'].sum())
        sell_cur_value.append(el[el['Тип'] == 'sell']['Количество'].sum())
        min_price.append(el[el['Тип'] == 'buy']['Цена'].min())
        max_price.append(el[el['Тип'] == 'sell']['Цена'].max())
        buy_value.append(el[el['Тип'] == 'buy']['Сумма'].sum())
        sell_value.append(el[el['Тип'] == 'sell']['Сумма'].sum())
        first_date.append(time_period_1(el)[0])
        last_date.append(time_period_1(el)[1])
        delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])
        
    else:
        if str(el['Тип']) == 'buy':
            buy_count_list.append(len(el))
            sell_count_list.append(0)
            buy_cur_value.append(el['Количество'].sum())
            sell_cur_value.append(0)
            min_price.append(el['Цена'].min())
            max_price.append(el['Цена'].max())
            buy_value.append(el['Сумма'].sum())
            sell_value.append(0)
            first_date.append(time_period_1(el)[0])
            last_date.append(time_period_1(el)[1])
            delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])
        else:
            buy_count_list.append(0)
            sell_count_list.append(len(el))
            buy_cur_value.append(0)
            sell_cur_value.append(el['Количество'].sum())
            min_price.append(el['Цена'].min())
            max_price.append(el['Цена'].max())
            buy_value.append(0)
            sell_value.append(el['Сумма'].sum())
            first_date.append(time_period_1(el)[0])
            last_date.append(time_period_1(el)[1])
            delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])

In [31]:
finish_df_06_2023 = pd.DataFrame({'cur_pair' : pair_list,
                          'trd_count' : trade_counts_list,
                          'sell_count': sell_count_list,
                          'buy_count': buy_count_list,
                          'all_cur_value': all_cur_value,
                          'buy_cur_value': buy_cur_value,
                          'sell_cur_value': sell_cur_value,
                          'min_price': min_price,
                          'max_price': max_price,
                          'buy_value': buy_value,
                          'sell_value': sell_value,
                          'first_date':  first_date, 
                          'last_date':  last_date,
                          'time_period': delta_date
                         },
                           columns=['cur_pair','trd_count', 'sell_count', 'buy_count', 'all_cur_value', 'buy_cur_value', 'sell_cur_value', 'min_price', 'max_price', 'buy_value', 'sell_value', 'first_date', 'last_date', 'time_period'])

ValueError: All arrays must be of the same length

In [32]:
finish_df_06_2023

,cur_pair,trd_count,sell_count,buy_count,all_cur_value,buy_cur_value,sell_cur_value,min_price,max_price,buy_value,sell_value,first_date,last_date,time_period
0,EXM_ETH,634,295,339,135581.184934,73161.089560,62420.095374,6.400000e-06,3.000000e-05,0.583311,0.511313,2021-05-05 22:52:00,2023-07-01 05:17:00,786 days 06:25:00
1,DOGE_USDT,19,8,11,1274.026319,887.232694,386.793625,4.000000e-02,9.000000e-02,65.280000,28.230358,2023-04-07 13:54:00,2023-07-01 01:20:00,84 days 11:26:00
2,DOGE_BTC,280,134,146,6729.375561,3364.167442,3365.208119,2.100000e-06,6.200000e-06,0.011697,0.013775,2022-12-05 09:31:00,2023-07-01 01:15:00,207 days 15:44:00
3,EXM_BTC,1350,644,706,388834.087079,206510.175074,182323.912005,3.900000e-07,2.500000e-06,0.157512,0.143389,2021-04-21 11:51:00,2023-07-01 01:05:00,800 days 13:14:00
4,WXT_BTC,432,190,242,186996.094617,96562.005261,90434.089355,7.000000e-08,5.000000e-07,0.012961,0.014626,2021-07-29 10:10:00,2023-06-30 21:14:00,701 days 11:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,ONT_BTC,1,1,0,23.993149,0.000000,23.993149,2.903000e-05,2.903000e-05,0.000000,0.000697,2021-06-04 10:46:00,2021-06-04 10:46:00,0 days 00:00:00
60,ONG_BTC,13,13,0,222.211205,0.000000,222.211205,2.000000e-05,2.178000e-05,0.000000,0.004740,2021-06-04 10:43:00,2021-06-04 10:44:00,0 days 00:01:00
61,XRP_USDT,2,1,1,17.325071,11.423863,5.901208,7.000000e-01,1.000000e+00,7.996704,5.901208,2021-05-23 12:44:00,2021-05-25 00:02:00,1 days 11:18:00
62,XEM_BTC,3,1,2,37.830499,27.153535,10.676964,4.500000e-06,7.000000e-06,0.000133,0.000075,2021-04-23 02:26:00,2021-05-23 16:45:00,30 days 14:19:00


In [33]:
# Загрузка библиотек и модулей

import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")
        
from scipy.stats import shapiro
from scipy.stats import probplot
from scipy.stats import ttest_ind, mannwhitneyu
from scipy.stats import chi2_contingency
# from statsmodels.stats.weightstats import zconfint

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

# Загрузка входных данных
df_first = pd.read_csv('trade-history-2021-11-07.csv')
df_middle = pd.read_csv('trade-history-2022-10-10.csv')
df_last = pd.read_csv('trade-history-2023-07-01.csv')

# Загрузка функций
def big_dataset(dataframe_first, dataframe_last):
    ### Concatinations dataframes
    return pd.concat([dataframe_first, dataframe_last], ignore_index=True).drop_duplicates()

big_df = pd.concat([df_last, pd.concat([df_middle, df_first], ignore_index=True).drop_duplicates()], ignore_index=True).drop_duplicates()
len(big_df)

def time_period(dataframe):
    all_dates = dataframe['Дата/время'].tolist()
    start = all_dates[-1]
    end = all_dates[0]
    start_dtm = datetime.datetime.strptime(start, '%d.%m.%Y %H:%M')
    end_dtm = datetime.datetime.strptime(end, '%d.%m.%Y %H:%M')
    print(f'\nПроизведён анализ временного периода торговли: с {start} по {end}'
        

  f'\nчто составляет {end_dtm - start_dtm}')
    
def all_pairs_list(dataframe):
    return dataframe['Валютная пара'].unique().tolist()

def df_curr_pair(dataframe, current_pair):
    current = str(current_pair.upper())
    return dataframe.loc[(dataframe['Валютная пара'] == current)]

def time_period_1(dataframe):
    all_dates = dataframe['Дата/время'].tolist()
    start = all_dates[-1]
    end = all_dates[0]
    start_dtm = datetime.datetime.strptime(start, '%d.%m.%Y %H:%M')
    end_dtm = datetime.datetime.strptime(end, '%d.%m.%Y %H:%M')
    return start_dtm, end_dtm
 
    
dataframe_pairs_list = []
for el in pair_list:
    dataframe_pairs_list.append(df_curr_pair(df_jun_2023, el))

pair_list = all_pairs_list(df_jun_2023)
trade_counts_list = []
buy_count_list = []
sell_count_list = []
all_cur_value = []
buy_cur_value = []
sell_cur_value = []
min_price = []
max_price = []
buy_value = []
sell_value = []
first_date = []
last_date = []
delta_date = []



for el in dataframe_pairs_list:
    trade_counts_list.append(len(el))
    all_cur_value.append(el['Количество'].sum())
    a = el['Тип'].value_counts()
    
    if len(a) == 2:
        buy_count_list.append(a[0])
        sell_count_list.append(a[1])  
        buy_cur_value.append(el[el['Тип'] == 'buy']['Количество'].sum())
        sell_cur_value.append(el[el['Тип'] == 'sell']['Количество'].sum())
        min_price.append(el[el['Тип'] == 'buy']['Цена'].min())
        max_price.append(el[el['Тип'] == 'sell']['Цена'].max())
        buy_value.append(el[el['Тип'] == 'buy']['Сумма'].sum())
        sell_value.append(el[el['Тип'] == 'sell']['Сумма'].sum())
        first_date.append(time_period_1(el)[0])
        last_date.append(time_period_1(el)[1])
        delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])
        
    else:
        if str(el['Тип']) == 'buy':
            buy_count_list.append(len(el))
            sell_count_list.append(0)
            buy_cur_value.append(el['Количество'].sum())
            sell_cur_value.append(0)
            min_price.append(el['Цена'].min())
            max_price.append(el['Цена'].max())
            buy_value.append(el['Сумма'].sum())
            sell_value.append(0)
            first_date.append(time_period_1(el)[0])
            last_date.append(time_period_1(el)[1])
            delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])
        else:
            buy_count_list.append(0)
            sell_count_list.append(len(el))
            buy_cur_value.append(0)
            sell_cur_value.append(el['Количество'].sum())
            min_price.append(el['Цена'].min())
            max_price.append(el['Цена'].max())
            buy_value.append(0)
            sell_value.append(el['Сумма'].sum())
            first_date.append(time_period_1(el)[0])
            last_date.append(time_period_1(el)[1])
            delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])

In [34]:
finish_df_06_2023 = pd.DataFrame({'cur_pair' : pair_list,
                          'trd_count' : trade_counts_list,
                          'sell_count': sell_count_list,
                          'buy_count': buy_count_list,
                          'all_cur_value': all_cur_value,
                          'buy_cur_value': buy_cur_value,
                          'sell_cur_value': sell_cur_value,
                          'min_price': min_price,
                          'max_price': max_price,
                          'buy_value': buy_value,
                          'sell_value': sell_value,
                          'first_date':  first_date, 
                          'last_date':  last_date,
                          'time_period': delta_date
                         },
                           columns=['cur_pair','trd_count', 'sell_count', 'buy_count', 'all_cur_value', 'buy_cur_value', 'sell_cur_value', 'min_price', 'max_price', 'buy_value', 'sell_value', 'first_date', 'last_date', 'time_period'])

In [35]:
finish_df_06_2023

,cur_pair,trd_count,sell_count,buy_count,all_cur_value,buy_cur_value,sell_cur_value,min_price,max_price,buy_value,sell_value,first_date,last_date,time_period
0,WXT_BTC,4,1,3,7.310806e+03,5.935806e+03,1.375000e+03,8.000000e-08,9.000000e-08,0.000520,0.000124,2023-06-28 14:16:00,2023-06-30 21:14:00,2 days 06:58:00
1,USDT_KZT,153,74,79,3.661356e+02,1.664454e+02,1.996902e+02,4.080000e+02,4.500000e+02,70989.824691,88014.426509,2023-06-02 06:54:00,2023-06-30 18:07:00,28 days 11:13:00
2,EXM_ETH,156,68,88,3.599832e+04,1.922693e+04,1.677140e+04,6.400000e-06,7.500000e-06,0.135345,0.120125,2023-06-01 00:21:00,2023-06-30 17:41:00,29 days 17:20:00
3,WXT_USDT,4,4,0,2.220000e+03,0.000000e+00,2.220000e+03,2.399990e-03,2.500000e-03,0.000000,5.527798,2023-06-30 10:52:00,2023-06-30 13:51:00,0 days 02:59:00
4,EXM_BTC,63,26,37,1.395859e+04,7.879713e+03,6.078875e+03,3.900000e-07,5.100000e-07,0.003525,0.002726,2023-06-02 08:35:00,2023-06-30 11:26:00,28 days 02:51:00
5,XLM_RUB,18,9,9,1.873606e+03,7.914477e+02,1.082158e+03,9.000000e+00,1.100000e+01,7168.846366,10601.384628,2023-06-24 09:44:00,2023-06-30 11:19:00,6 days 01:35:00
6,XRP_ETH,19,8,11,1.733523e+02,1.013340e+02,7.201833e+01,2.500000e-04,3.200000e-04,0.028295,0.021466,2023-06-04 15:27:00,2023-06-28 19:30:00,24 days 04:03:00
7,DOGE_BTC,15,4,11,7.273943e+02,5.247478e+02,2.026464e+02,2.100000e-06,2.700000e-06,0.001236,0.000507,2023-06-05 16:01:00,2023-06-28 19:21:00,23 days 03:20:00
8,DAI_RUB,16,5,11,6.560645e+01,2.293709e+01,4.266936e+01,8.300000e+01,9.500000e+01,1931.598170,3817.262756,2023-06-13 01:03:00,2023-06-28 13:40:00,15 days 12:37:00
9,TON_BTC,10,3,7,1.846693e+02,1.350713e+02,4.959802e+01,4.500000e-05,6.000000e-05,0.006535,0.002812,2023-06-10 04:50:00,2023-06-28 02:49:00,17 days 21:59:00


In [36]:
finish_df_06_2023['trd_count'].sum()

565

In [37]:
finish_df_06_2023['buy_count'].sum()

316

In [38]:
finish_df_06_2023['sell_count'].sum()

249

In [54]:
# Загрузка библиотек и модулей

import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")
        
from scipy.stats import shapiro
from scipy.stats import probplot
from scipy.stats import ttest_ind, mannwhitneyu
from scipy.stats import chi2_contingency
# from statsmodels.stats.weightstats import zconfint

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

# Загрузка входных данных
df_first = pd.read_csv('trade-history-2021-11-07.csv')
df_middle = pd.read_csv('trade-history-2022-10-10.csv')
df_last = pd.read_csv('trade-history-2023-07-01.csv')

# Загрузка функций
def big_dataset(dataframe_first, dataframe_last):
    ### Concatinations dataframes
    return pd.concat([dataframe_first, dataframe_last], ignore_index=True).drop_duplicates()

big_df = pd.concat([df_last, pd.concat([df_middle, df_first], ignore_index=True).drop_duplicates()], ignore_index=True).drop_duplicates()
len(big_df)

def time_period(dataframe):
    all_dates = dataframe['Дата/время'].tolist()
    start = all_dates[-1]
    end = all_dates[0]
    start_dtm = datetime.datetime.strptime(start, '%d.%m.%Y %H:%M')
    end_dtm = datetime.datetime.strptime(end, '%d.%m.%Y %H:%M')
    print(f'\nПроизведён анализ временного периода торговли: с {start} по {end}'
        

  f'\nчто составляет {end_dtm - start_dtm}')
    
def all_pairs_list(dataframe):
    return dataframe['Валютная пара'].unique().tolist()

def df_curr_pair(dataframe, current_pair):
    current = str(current_pair.upper())
    return dataframe.loc[(dataframe['Валютная пара'] == current)]

def time_period_1(dataframe):
    all_dates = dataframe['Дата/время'].tolist()
    start = all_dates[-1]
    end = all_dates[0]
    start_dtm = datetime.datetime.strptime(start, '%d.%m.%Y %H:%M')
    end_dtm = datetime.datetime.strptime(end, '%d.%m.%Y %H:%M')
    return start_dtm, end_dtm
 
    
dataframe_pairs_list = []
for el in pair_list:
    dataframe_pairs_list.append(df_curr_pair(df_jun_2023, el))

pair_list = all_pairs_list(df_jun_2023)
trade_counts_list = []
buy_count_list = []
sell_count_list = []
delta_sell_buy = []
all_cur_value = []
buy_cur_value = []
sell_cur_value = []
delta_cuur_value = []
min_price = []
max_price = []
buy_value = []
sell_value = []
profit = []
first_date = []
last_date = []
delta_date = []



for el in dataframe_pairs_list:
    trade_counts_list.append(len(el))
    all_cur_value.append(el['Количество'].sum())
    a = el['Тип'].value_counts()
    
    if len(a) == 2:
        buy_count_list.append(a[0])
        sell_count_list.append(a[1])  
        buy_cur_value.append(el[el['Тип'] == 'buy']['Количество'].sum())
        sell_cur_value.append(el[el['Тип'] == 'sell']['Количество'].sum())
        min_price.append(el[el['Тип'] == 'buy']['Цена'].min())
        max_price.append(el[el['Тип'] == 'sell']['Цена'].max())
        buy_value.append(el[el['Тип'] == 'buy']['Сумма'].sum())
        sell_value.append(el[el['Тип'] == 'sell']['Сумма'].sum())
        first_date.append(time_period_1(el)[0])
        last_date.append(time_period_1(el)[1])
        delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])
        delta_sell_buy.append(a[1] - a[0])
        delta_cuur_value.append(el[el['Тип'] == 'sell']['Количество'].sum() - el[el['Тип'] == 'buy']['Количество'].sum())
        profit.append(el[el['Тип'] == 'sell']['Сумма'].sum() - el[el['Тип'] == 'buy']['Сумма'].sum())
    else:
        if str(el['Тип']) == 'buy':
            buy_count_list.append(len(el))
            sell_count_list.append(0)
            buy_cur_value.append(el['Количество'].sum())
            sell_cur_value.append(0)
            min_price.append(el['Цена'].min())
            max_price.append(el['Цена'].max())
            buy_value.append(el['Сумма'].sum())
            sell_value.append(0)
            first_date.append(time_period_1(el)[0])
            last_date.append(time_period_1(el)[1])
            delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])
            delta_sell_buy.append(0 - len(el))
            delta_cuur_value.append(el[el['Тип'] == 'sell']['Количество'].sum() - el[el['Тип'] == 'buy']['Количество'].sum())
            profit.append(0 - el['Сумма'].sum())
        else:
            buy_count_list.append(0)
            sell_count_list.append(len(el))
            buy_cur_value.append(0)
            sell_cur_value.append(el['Количество'].sum())
            min_price.append(el['Цена'].min())
            max_price.append(el['Цена'].max())
            buy_value.append(0)
            sell_value.append(el['Сумма'].sum())
            first_date.append(time_period_1(el)[0])
            last_date.append(time_period_1(el)[1])
            delta_date.append(time_period_1(el)[1] - time_period_1(el)[0])
            delta_sell_buy.append(len(el))
            delta_cuur_value.append(el[el['Тип'] == 'sell']['Количество'].sum() - el[el['Тип'] == 'buy']['Количество'].sum())
            profit.append(el['Сумма'].sum())

In [63]:
finish_df_06_2023 = pd.DataFrame({'cur_pair' : pair_list,
                          'trd_count' : trade_counts_list,
                          'buy_count': buy_count_list,
                          'sell_count': sell_count_list,
                          'delta_sell_buy': delta_sell_buy,
                          'all_cur_value': all_cur_value,
                          'buy_cur_value': buy_cur_value,
                          'sell_cur_value': sell_cur_value,
                          'delta_cur_value': delta_cuur_value,
                          'min_price': min_price,
                          'max_price': max_price,
                          'buy_value': buy_value,
                          'sell_value': sell_value,
                          'profit': profit,
                          'first_date':  first_date, 
                          'last_date':  last_date,
                          'time_period': delta_date
                         },
                           columns=['cur_pair','trd_count', 'buy_count', 'sell_count', 'delta_sell_buy', 'all_cur_value', 'buy_cur_value', 'sell_cur_value', 'delta_cur_value', 'min_price', 'max_price', 'buy_value', 'sell_value', 'profit', 'first_date', 'last_date', 'time_period'])

In [56]:
finish_df_06_2023

,cur_pair,trd_count,sell_count,buy_count,delta_sell_buy,all_cur_value,buy_cur_value,delta_cuur_value,sell_cur_value,min_price,max_price,buy_value,sell_value,profit,first_date,last_date,time_period
0,WXT_BTC,4,1,3,-2,7.310806e+03,5.935806e+03,-4.560806e+03,1.375000e+03,8.000000e-08,9.000000e-08,0.000520,0.000124,-0.000397,2023-06-28 14:16:00,2023-06-30 21:14:00,2 days 06:58:00
1,USDT_KZT,153,74,79,-5,3.661356e+02,1.664454e+02,3.324487e+01,1.996902e+02,4.080000e+02,4.500000e+02,70989.824691,88014.426509,17024.601818,2023-06-02 06:54:00,2023-06-30 18:07:00,28 days 11:13:00
2,EXM_ETH,156,68,88,-20,3.599832e+04,1.922693e+04,-2.455532e+03,1.677140e+04,6.400000e-06,7.500000e-06,0.135345,0.120125,-0.015220,2023-06-01 00:21:00,2023-06-30 17:41:00,29 days 17:20:00
3,WXT_USDT,4,4,0,4,2.220000e+03,0.000000e+00,-2.220000e+03,2.220000e+03,2.399990e-03,2.500000e-03,0.000000,5.527798,5.527798,2023-06-30 10:52:00,2023-06-30 13:51:00,0 days 02:59:00
4,EXM_BTC,63,26,37,-11,1.395859e+04,7.879713e+03,-1.800838e+03,6.078875e+03,3.900000e-07,5.100000e-07,0.003525,0.002726,-0.000799,2023-06-02 08:35:00,2023-06-30 11:26:00,28 days 02:51:00
5,XLM_RUB,18,9,9,0,1.873606e+03,7.914477e+02,2.907102e+02,1.082158e+03,9.000000e+00,1.100000e+01,7168.846366,10601.384628,3432.538262,2023-06-24 09:44:00,2023-06-30 11:19:00,6 days 01:35:00
6,XRP_ETH,19,8,11,-3,1.733523e+02,1.013340e+02,-2.931563e+01,7.201833e+01,2.500000e-04,3.200000e-04,0.028295,0.021466,-0.006830,2023-06-04 15:27:00,2023-06-28 19:30:00,24 days 04:03:00
7,DOGE_BTC,15,4,11,-7,7.273943e+02,5.247478e+02,-3.221014e+02,2.026464e+02,2.100000e-06,2.700000e-06,0.001236,0.000507,-0.000729,2023-06-05 16:01:00,2023-06-28 19:21:00,23 days 03:20:00
8,DAI_RUB,16,5,11,-6,6.560645e+01,2.293709e+01,1.973227e+01,4.266936e+01,8.300000e+01,9.500000e+01,1931.598170,3817.262756,1885.664586,2023-06-13 01:03:00,2023-06-28 13:40:00,15 days 12:37:00
9,TON_BTC,10,3,7,-4,1.846693e+02,1.350713e+02,-8.547324e+01,4.959802e+01,4.500000e-05,6.000000e-05,0.006535,0.002812,-0.003722,2023-06-10 04:50:00,2023-06-28 02:49:00,17 days 21:59:00


In [64]:
finish_df_06_2023[finish_df_06_2023['profit'] < 0]

,cur_pair,trd_count,buy_count,sell_count,delta_sell_buy,all_cur_value,buy_cur_value,sell_cur_value,delta_cur_value,min_price,max_price,buy_value,sell_value,profit,first_date,last_date,time_period
0,WXT_BTC,4,3,1,-2,7310.805556,5935.805556,1375.000000,-4560.805556,8.000000e-08,9.000000e-08,0.000520,0.000124,-0.000397,2023-06-28 14:16:00,2023-06-30 21:14:00,2 days 06:58:00
2,EXM_ETH,156,88,68,-20,35998.322556,19226.927461,16771.395094,-2455.532367,6.400000e-06,7.500000e-06,0.135345,0.120125,-0.015220,2023-06-01 00:21:00,2023-06-30 17:41:00,29 days 17:20:00
4,EXM_BTC,63,37,26,-11,13958.587782,7879.712856,6078.874926,-1800.837930,3.900000e-07,5.100000e-07,0.003525,0.002726,-0.000799,2023-06-02 08:35:00,2023-06-30 11:26:00,28 days 02:51:00
6,XRP_ETH,19,11,8,-3,173.352294,101.333964,72.018329,-29.315635,2.500000e-04,3.200000e-04,0.028295,0.021466,-0.006830,2023-06-04 15:27:00,2023-06-28 19:30:00,24 days 04:03:00
7,DOGE_BTC,15,11,4,-7,727.394271,524.747837,202.646434,-322.101403,2.100000e-06,2.700000e-06,0.001236,0.000507,-0.000729,2023-06-05 16:01:00,2023-06-28 19:21:00,23 days 03:20:00
9,TON_BTC,10,7,3,-4,184.669276,135.071256,49.598020,-85.473235,4.500000e-05,6.000000e-05,0.006535,0.002812,-0.003722,2023-06-10 04:50:00,2023-06-28 02:49:00,17 days 21:59:00
11,XTZ_RUB,56,31,25,-6,152.186374,87.706156,64.480217,-23.225939,5.300000e+01,8.300000e+01,5499.526416,4608.257820,-891.268595,2023-06-03 01:34:00,2023-06-24 11:44:00,21 days 10:10:00
15,DOGE_USDT,8,5,3,-2,449.593000,246.500000,203.093000,-43.407000,4.000000e-02,7.000000e-02,12.570000,12.555582,-0.014418,2023-06-10 10:06:00,2023-06-11 08:51:00,0 days 22:45:00
16,DOGE_RUB,11,8,3,-5,189.840278,121.221100,68.619178,-52.601922,5.600000e+00,6.800000e+00,688.790000,458.975565,-229.814435,2023-06-01 11:02:00,2023-06-10 18:06:00,9 days 07:04:00


# Результирующая Таблица по валютам

In [82]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")
        
from scipy.stats import shapiro
from scipy.stats import probplot
from scipy.stats import ttest_ind, mannwhitneyu
from scipy.stats import chi2_contingency
# from statsmodels.stats.weightstats import zconfint

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline


df_first = pd.read_csv('trade-history-2021-11-07.csv')
df_middle = pd.read_csv('trade-history-2022-10-10.csv')
df_last = pd.read_csv('trade-history-2023-07-01.csv')

def big_dataset(dataframe_first, dataframe_last):
    ### Concatinations dataframes
    return pd.concat([dataframe_first, dataframe_last], ignore_index=True).drop_duplicates()

def time_period(dataframe):
    all_dates = dataframe['Дата/время'].tolist()
    start = all_dates[-1]
    end = all_dates[0]
    start_dtm = datetime.datetime.strptime(start, '%d.%m.%Y %H:%M')
    end_dtm = datetime.datetime.strptime(end, '%d.%m.%Y %H:%M')
    print(f'\nПроизведён анализ временного периода торговли: с {start} по {end}'
        

  f'\nчто составляет {end_dtm - start_dtm}')
    
def all_pairs_list(dataframe):
    return dataframe['Валютная пара'].unique().tolist()
    
def currents(pair_list):
    currents_list = []
    for el in pair_list:
        curs = el.split('_')
        for el in curs:
            currents_list.append(el)
    return set(currents_list)


big_df = pd.concat([df_last, pd.concat([df_middle, df_first], ignore_index=True).drop_duplicates()], ignore_index=True).drop_duplicates()
len(big_df)

pair_list = all_pairs_list(big_df)
currents_list = currents(pair_list)

currents = []
pair_counts = []
currents_pairs = []

for el in currents_list:
    currents.append(el)
    i = 0
    pairs = []
    for cur_pair in pair_list :
        if el in cur_pair:
            i += 1
            pairs.append(cur_pair)
    pair_counts.append(i)
    currents_pairs.append(pairs)

    

In [83]:
print(len(currents), len(pair_counts), len(currents_pairs))

37 37 37


In [87]:
currents_df = pd.DataFrame({'currents' : currents,
                          'pair_counts' : pair_counts,
                          'current_pairs': currents_pairs
                         },
                           columns=['currents', 'pair_counts', 'current_pairs'])

In [88]:
currents_df

,currents,pair_counts,current_pairs
0,XRP,7,"[XRP_ETH, XRP_GBP, XRP_RUB, XRP_USD, XRP_UAH, ..."
1,KZT,1,[USDT_KZT]
2,ONT,1,[ONT_BTC]
3,ALGO,2,"[ALGO_BTC, ALGO_RUB]"
4,SHIB,3,"[SHIB_RUB, SHIB_UAH, SHIB_USDT]"
5,TONCOIN,1,[TONCOIN_BTC]
6,ETH,7,"[EXM_ETH, XRP_ETH, CRON_ETH, DAI_ETH, ETH_GBP,..."
7,PEPE,1,[PEPE_USDT]
8,XEM,1,[XEM_BTC]
9,GBP,4,"[ETH_GBP, DOGE_GBP, XRP_GBP, USDT_GBP]"


In [153]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")
        
from scipy.stats import shapiro
from scipy.stats import probplot
from scipy.stats import ttest_ind, mannwhitneyu
from scipy.stats import chi2_contingency
# from statsmodels.stats.weightstats import zconfint

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline


df_first = pd.read_csv('trade-history-2021-11-07.csv')
df_middle = pd.read_csv('trade-history-2022-10-10.csv')
df_last = pd.read_csv('trade-history-2023-07-01.csv')

def big_dataset(dataframe_first, dataframe_last):
    ### Concatinations dataframes
    return pd.concat([dataframe_first, dataframe_last], ignore_index=True).drop_duplicates()

def time_period(dataframe):
    all_dates = dataframe['Дата/время'].tolist()
    start = all_dates[-1]
    end = all_dates[0]
    start_dtm = datetime.datetime.strptime(start, '%d.%m.%Y %H:%M')
    end_dtm = datetime.datetime.strptime(end, '%d.%m.%Y %H:%M')
    print(f'\nПроизведён анализ временного периода торговли: с {start} по {end}'
        

  f'\nчто составляет {end_dtm - start_dtm}')
    
def all_pairs_list(dataframe):
    return dataframe['Валютная пара'].unique().tolist()
    
def currents(pair_list):
    currents_list = []
    for el in pair_list:
        curs = el.split('_')
        for el in curs:
            currents_list.append(el)
    return set(currents_list)


big_df = pd.concat([df_last, pd.concat([df_middle, df_first], ignore_index=True).drop_duplicates()], ignore_index=True).drop_duplicates()
len(big_df)

pair_list = all_pairs_list(big_df)
currents_list = currents(pair_list)

currents = []
pair_counts = []
currents_pairs = []
vol = []
vol_buy = []
vol_sell = []
profit_vol = []

for el in currents_list:
    currents.append(el)
    i = 0
    pairs = []
    for cur_pair in pair_list :
        if el in cur_pair:
            i += 1
            pairs.append(cur_pair)
    pair_counts.append(i)
    currents_pairs.append(pairs)

    df_current_list = []
    for cur_pair in pairs:
         df_current_list.append(df_curr_pair(big_df, cur_pair))
    
    current_buy_value = []
    current_sell_value = []
    current_value = []
    for df in df_current_list:   
            split_pair = str(set(df['Валютная пара'])).replace("{'", '').replace("'}", '').split('_')     
            if el == split_pair[0]:
                current_value.append(df['Количество'].sum())
                current_buy_value.append(df[df['Тип'] == 'buy']['Количество'].sum())
                current_sell_value.append(df[df['Тип'] == 'sell']['Количество'].sum())
            else:
                current_value.append(df['Сумма'].sum())
                current_buy_value.append(df[df['Тип'] == 'buy']['Сумма'].sum())
                current_sell_value.append(df[df['Тип'] == 'sell']['Сумма'].sum())
            
    vol.append(sum(current_value))
    vol_buy.append(sum(current_buy_value))
    vol_sell.append(sum(current_sell_value))
    profit_vol.append(sum(current_sell_value) - sum(current_buy_value))

In [155]:
currents_df = pd.DataFrame({'currents' : currents,
                          'pair_counts' : pair_counts,
                          'current_pairs': currents_pairs,
                          'volue': vol,
                          'buy_vol': vol_buy, 
                          'sell_vol': vol_sell,
                          'vol_profit': profit_vol
                        },
                           columns=['currents', 'pair_counts', 'current_pairs', 'volue', 'buy_vol', 'sell_vol', 'vol_profit'])

In [156]:
currents_df

,currents,pair_counts,current_pairs,volue,buy_vol,sell_vol,vol_profit
0,XRP,7,"[XRP_ETH, XRP_GBP, XRP_RUB, XRP_USD, XRP_UAH, ...",5.462026e+03,2.977343e+03,2.484683e+03,-4.926609e+02
1,KZT,1,[USDT_KZT],1.336945e+06,6.513244e+05,6.856210e+05,3.429659e+04
2,ONT,1,[ONT_BTC],2.399315e+01,0.000000e+00,2.399315e+01,2.399315e+01
3,ALGO,2,"[ALGO_BTC, ALGO_RUB]",6.120736e+02,2.854413e+02,3.266323e+02,4.119104e+01
4,SHIB,3,"[SHIB_RUB, SHIB_UAH, SHIB_USDT]",4.170091e+08,2.287382e+08,1.882709e+08,-4.046723e+07
5,TONCOIN,1,[TONCOIN_BTC],2.481336e+02,1.342529e+02,1.138807e+02,-2.037220e+01
6,ETH,7,"[EXM_ETH, XRP_ETH, CRON_ETH, DAI_ETH, ETH_GBP,...",3.865448e+00,1.987338e+00,1.878110e+00,-1.092283e-01
7,PEPE,1,[PEPE_USDT],2.899609e+07,1.789504e+07,1.110105e+07,-6.793989e+06
8,XEM,1,[XEM_BTC],3.783050e+01,2.715354e+01,1.067696e+01,-1.647657e+01
9,GBP,4,"[ETH_GBP, DOGE_GBP, XRP_GBP, USDT_GBP]",5.766170e+03,2.962178e+03,2.803993e+03,-1.581847e+02
